In [2]:
# Exploration
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px

# Model Implementation
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose

__Methodology:__
1. Data Exploration and Preprocessing: We start by exploring the provided dataset, gaining a deep understanding of the available features, their relationships, and potential challenges. Through careful preprocessing, we handle missing values, encode categorical variables, and prepare the data for further analysis.

2. Exploratory Data Analysis (EDA): Armed with a rich dataset, we embark on an exploratory journey to uncover hidden patterns, trends, and correlations. Engaging visualizations and statistical techniques help us understand the key factors influencing sales, detect seasonal variations, and identify any intriguing trends that may influence business decisions.

3. Time Series Analysis: Equipped with a solid foundation from EDA, we employ time series analysis techniques to capture the temporal patterns within the sales data. Leveraging the power of models like ARIMA (Autoregressive Integrated Moving Average), we aim to build a forecasting model that can capture the complex dynamics of retail sales.

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./train.csv')
stores = pd.read_csv('./stores.csv')
features = pd.read_csv('./features.csv')

### __Data Exploration__

In [12]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [14]:
train.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


In [32]:
display(stores.head())
print('\n')
display(stores.info())

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


None

In [25]:
print('Missing Values: ')
print(train.isnull().sum())

Missing Values: 
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64


In [31]:
display(features.head())
display(features.info())

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


None

##### __Explore Unique Values__

In [37]:
# Explore unique values in categorical variables
print('\nUnique Store Types: ')
print(stores['Type'].unique())


Unique Store Types: 
['A' 'B' 'C']


In [54]:
print('\nUnique Store Types: ')
print(train['Dept'].sort_values(ascending=True).unique())


Unique Store Types: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 54 55 56 58 59 60 65 67 71 72 74 77 78 79 80 81 82 83 85 87 90
 91 92 93 94 95 96 97 98 99]


In [55]:
print('\nUnique Holidays: ')
print(train['IsHoliday'].unique())


Unique Holidays: 
[False  True]


### __Feature Engineering__
1. Combining store and department information: The 'Store' and 'Dept' columns are combined into a new column called 'Store_Dept' by concatenating their values as strings. This allows for better representation of the relationship between store and department in the dataset.

2. Extracting month and year from the Date column: The 'Date' column is parsed into separate columns for month and year. This allows for easier analysis and grouping of data based on these time components.

3. Calculating the total markdown amount: The individual 'MarkDown' columns are used to calculate the total markdown amount for each record. This aggregation provides a consolidated view of the promotional markdowns for a specific store and department.

4. Encoding categorical variables: Categorical variables, such as the 'IsHoliday' column, are encoded to numerical format. In the example provided, the 'IsHoliday' column is converted to integer values (0 for False and 1 for True). This enables the use of these variables in machine learning algorithms that require numeric input.

5. Merging additional features to the train and test datasets: The 'stores' and 'features' datasets are merged with the train and test datasets based on the common columns 'Store' and 'Date'. This combines the relevant additional information (such as store type, size, temperature, fuel price, etc.) with the corresponding records in the train and test datasets.

In [63]:
# Combine store and department information
train['Store_Dept'] = train['Store'].astype(str) + '_' + train['Dept'].astype(str)
test['Store_Dept'] = test['Store'].astype(str) + '_' + test['Dept'].astype(str)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Store_Dept
0,1,1,2010-02-05,24924.50,False,1_1
1,1,1,2010-02-12,46039.49,True,1_1
2,1,1,2010-02-19,41595.55,False,1_1
3,1,1,2010-02-26,19403.54,False,1_1
4,1,1,2010-03-05,21827.90,False,1_1
...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,45_98
421566,45,98,2012-10-05,628.10,False,45_98
421567,45,98,2012-10-12,1061.02,False,45_98
421568,45,98,2012-10-19,760.01,False,45_98
